In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

# Load data from CSV files
farms = pd.read_csv('/Users/allenyang/Downloads/Schulich/Omis6000/Assignment1/farms.csv')
processing = pd.read_csv('/Users/allenyang/Downloads/Schulich/Omis6000/Assignment1/processing.csv')
centers = pd.read_csv('/Users/allenyang/Downloads/Schulich/Omis6000/Assignment1/centers.csv')

# Print column names to verify
# print("Farms columns:", farms.columns)
# print("Processing columns:", processing.columns)
# print("Centers columns:", centers.columns)


# Select a subset of the data to reduce the model size
farms_subset = farms.head(5)  # Select first 5 farms
processing_subset = processing.head(5)  # Select first 5 processing plants
centers_subset = centers.head(5)  # Select first 5 centers

# Create the optimization model
model = gp.Model("BioAgri Solutions")

# Decision variables for transportation from farms to processing plants
x = model.addVars(len(farms_subset), len(processing_subset), lb=0, vtype=GRB.CONTINUOUS, name="Transport_Farm_to_Plant")

# Decision variables for transportation from processing plants to home centers
y = model.addVars(len(processing_subset), len(centers_subset), lb=0, vtype=GRB.CONTINUOUS, name="Transport_Plant_to_Center")

# Objective function: Minimize total cost
total_cost = gp.quicksum(farms_subset.loc[i, 'Cost_Per_Ton'] * x[i, j] + 
                         farms_subset.iloc[i, 4 + j] * x[i, j] +  # Transport cost from farm to plant
                         processing_subset.loc[j, 'Processing_Cost_Per_Ton'] * x[i, j] + 
                         processing_subset.iloc[j, 4 + k] * y[j, k]  # Transport cost from plant to center
                         for i in range(len(farms_subset)) 
                         for j in range(len(processing_subset)) 
                         for k in range(len(centers_subset)))

model.setObjective(total_cost, GRB.MINIMIZE)

# Supply constraints for farms
for i in range(len(farms_subset)):
    model.addConstr(gp.quicksum(x[i, j] for j in range(len(processing_subset))) <= farms_subset.loc[i, 'Bio_Material_Capacity_Tons'], name=f"Supply_Constraint_Farm_{i}")

# Capacity constraints for processing plants
for j in range(len(processing_subset)):
    model.addConstr(gp.quicksum(x[i, j] for i in range(len(farms_subset))) <= processing_subset.loc[j, 'Capacity_Tons'], name=f"Capacity_Constraint_Plant_{j}")

# Demand constraints for home centers
for k in range(len(centers_subset)):
    model.addConstr(gp.quicksum(y[j, k] for j in range(len(processing_subset))))

# Optimize the model
model.optimize()

# Check if the optimization was successful
if model.status == GRB.OPTIMAL:
    print("Optimal Solution Found")
    print("Total Cost: ", model.objVal)
else:
    print("No feasible solution found.")

Set parameter Username
Set parameter LicenseID to value 2610025
Academic license - for non-commercial use only - expires 2026-01-14


GurobiError: Invalid value for Model.addConstr